Ideas:
1. Improve evaluation function (should this be given by default, or should the student improve it? You can just slap an engine on there too, btw)
2. Explanations (Maybe a commentator that lists the virtues of each move or something? Looks at all the possible moves from your given position and can check if you minimax-ed right?)

In [1]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

MAX_DEPTH = 4
rand = random.Random(0)

In [2]:
SCORES = {
    chess.KING : 200,
    chess.QUEEN : 9,
    chess.ROOK : 5,
    chess.BISHOP : 3,
    chess.KNIGHT : 3,
    chess.PAWN : 1,
}

# Very basic eval function based on counting pieces on the board
def evaluation(b:chess.Board):
    if b.is_checkmate():
        return -inf if b.turn == chess.WHITE else inf
    
    # The list needs to be size 7 because PIECE_TYPES goes from 1 to 6
    piece_count = { c : [0] * 7 for c in chess.COLORS }
    for piece in b.piece_map().values():
        piece_count[piece.color][piece.piece_type] += 1

    total = 0
    for piece in chess.PIECE_TYPES:
        diff = piece_count[chess.WHITE][piece] - piece_count[chess.BLACK][piece]
        total += SCORES[piece] * diff
    
    return total

def get_alpha_beta_move(board:chess.Board, player):
    opponent = not player

    def max_value(b:chess.Board, alpha, beta, depth):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        v = -inf
        move = None
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            s_val = min_value(temp, alpha, beta, depth + 1)
            if s_val > v:
                v, move = s_val, m
            if v >= beta:
                return v, move
            alpha = max(alpha, v)
        return v, move

    def min_value(b:chess.Board, alpha, beta, depth):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e
        
        v = inf
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            v = min(v, max_value(temp, alpha, beta, depth + 1)[0])
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    return max_value(board, -inf, inf, 0)[1]

def get_expectimax_move(board:chess.Board, player):
    opponent = not player

    def max_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        v = -inf
        move = None
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            s_val = chance_value(temp, depth + 1)
            if s_val > v:
                v, move = s_val, m
        return v, move

    def chance_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        successors = []
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            successors.append(max_value(temp, depth + 1)[0])
        return sum(successors) / len(successors)
    
    return max_value(board, 0)[1]

def get_random_move(b:chess.Board, player):
    return rand.choice(list(b.legal_moves))

In [3]:
def play_game(p1, p2):
    """
    Play a game of chess

    Parameters p1 and p2 represent the white and black players respectively

    They should be one of:

    get_alpha_beta_move,
    get_expectimax_move,
    get_random_move

    Turns must be progressed manually by pressing Enter when prompted
    
    Enter "q" to stop the game
    """
    board = chess.Board()
    display(board)
    current_player = p1

    while not board.is_checkmate():
        move = current_player(board, board.turn)
        board.push(move)
        clear_output(True)
        display(board)
        current_player = p1 if board.turn == chess.WHITE else p2
        if input() == "q":
            break

In [ ]:
play_game(get_alpha_beta_move, get_alpha_beta_move)

In [ ]:
play_game(get_expectimax_move, get_random_move)